# テキスト分類器の学習
予測までの流れとして、

トークンエンコーディング > トークン埋め込み > エンコーダ > 隠れ層 > 分類ヘッド > 予測

となる。  
※今までのがトークンエンコーディング  

事項からは、DistilBERTで以下両方を検討し、そのトレードオフを検証する  
- 特徴抽出
  - 隠れ状態を特徴として利用し、分類器を学習する。事前学習済みモデルは更新しない。
- ファインチューニング
  - モデル全体をend-to-endで学習する。事前学習済みモデルも同時に更新する。

## 特徴抽出器としてのTransformer

ボディの重みを変えず、分類器のみを学習する。  
GPUなしで素早く学習できるが、浅いモデルとなり、勾配に依存するモデルには不向き。

In [3]:
from transformers import AutoModel
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)
# model_ckptを指定し、指定したデバイスへモデルを読み込んでいる。

In [6]:
# TIPS: TensorFlowあるいはPyTorchの重みのみリリースされている場合、フレームワーク間の変換が可能
# 以下はPyTorchのみしかリリースされていないxlm-roberta-baseをTensorFlowへ変換する例
cache = """
from transformers import TFAutoModel

tf_model = TFAutoModel.from_pretrained(model_ckpt)
tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base")
tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base", from_pt=True)
"""

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [26]:
text = """this is a test"""
inputs = tokenizer(text, return_tensors="pt")
# return_tensors ... PyTorchテンソルに変換する。必須。

print(f"Input tensor shape: {inputs['input_ids'].size()}")
# テンソルは [batch_size, n_tokens] という形状で得られる。

inputs
# inputs.items()

Input tensor shape: torch.Size([1, 6])


{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [27]:
for k, v in inputs.items():
    print(k, v)    

input_ids tensor([[ 101, 2023, 2003, 1037, 3231,  102]])
attention_mask tensor([[1, 1, 1, 1, 1, 1]])


In [28]:
# modelと同じdeviceに、PyTorchテンソルを設置する。
inputs = {k:v.to(device) for k, v in inputs.items()} # items() ... Dict型の要素をキーと値ペアの配列に変換する。順不同。
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231,  102]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [29]:
with torch.no_grad():
    outputs = model(**inputs)
outputs
# torch.no_grad()は勾配の自動計算を無効にしている。 計算のメモリ使用量を減らす事が出来る。

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [30]:
# outputsには 隠れ状態、損失、アテンション等のオブジェクトが含まれる
# 現在のモデルには、最後の隠れ状態である１属性のみを返す
outputs.last_hidden_state.size()
# [batch_size, n_tokens, hidden_dim] という形をしている。
# ∴1バッチ、6個の入力トークン、768次元のベクトルということになる。

torch.Size([1, 6, 768])

In [33]:
# 分類タスクの場合、入力特徴として[CLS]トークンに関連付けられた隠れ状態だけを使うのが一般的
# これは各系列の最初に現れるので、このように抽出可能
outputs.last_hidden_state[:, 0].size() # = [:][0]の意、配列指定の:はすべての行の取得、カンマは次元の区切りを表す。= [:, 0, :]

torch.Size([1, 768])

In [35]:
# これまでの処理をラップ
def extract_hidden_states(batch):
    # モデルの入力をGPU上に配置
    inputs = {k:v.to(device) for k,v in batch.items()
             if k in tokenizer.model_input_names} # 今回の場合、input_idsやattention_maskが含まれていることが条件
    # 最後の隠れ状態を抽出
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state #**inputsは可変長キーワード引数を表す。input_ids, attention_maskなどのキーをまとめて渡せる。
    # [CLS]トークンに対するベクトルを返す
    return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [37]:
from datasets import load_dataset

emotions = load_dataset("dair-ai/emotion")

In [40]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [44]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [45]:
# input_ids と attension_maskの列をtorch形式に変換
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [46]:
# 隠れ状態を一度に抽出（予測）
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)
emotions_hidden

Map: 100%|██████████| 2000/2000 [00:02<00:00, 802.01 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
})

In [ ]:
emotions_hidden["train"].column_names